![alt text](images/lol.png "Title")





https://www.leagueoflegends.com/en-us/how-to-play/

### Détails du sujet

Ce jeu de données est un ensemble de data de League of Legends sur les matchs compétitifs. 
Si vous ne connaissez pas League of Legends, il s'agit d'un jeu où deux équipes de 5 joueurs s'affrontent. Chaque joueur choisi un champion, qui est un personnage parmi une liste. La partie d'arrête quand une équipe a détruit le nexus adverse. 

Le jeu expliqué en trois minutes : https://www.youtube.com/watch?v=OfYU4gbk13w

Chargez le dataset et répondez aux questions sous forme de phrase. 
Vous pouvez utiliser print(f"{}") pour rédiger des réponses complètes. 

Exemple : 

resultat = NombredeMort()

print(f"Le nombre de mort dans la partie est de {resultat}.")



Si vous avez des questions : 

                - Discord : @T.Bellanger (Salon Aide Prof sur le serveur 3A)

                - Teams : Thibaut Bellanger

                - Mail : thibaut.bellanger@esiea.fr

##### Import

In [1]:
#import

# Découverte du dataset

#### Chargement du dataset

#### Afficher les 2 premières lignes

#### Il y a-t-il des valeurs manquantes ? 

#### Il y a-t-il des doublons ? 

#### Ce dataset a une forme particulière. Le champ Address sert d'ID unique pour chaque partie. 

#### Combien de parties différentes contient ce dataset ? 

    Vous devez répondre avec une phrase pour chaque question, n'affichez pas seulement un résultat, répondez à la question !

Au cours d'une partie, plusieurs évènements se produisent. Les joueurs s'éliminent plusieurs fois par exemple. Nous ne pouvons pas savoir à l'avance combien de "Victim" nous aurons dans une partie, et ce pour chaque partie, donc les lignes sont multipliées. 

Voici un exemple: 
    
| Partie_ID  | Victim          | Killer | Time_kills|
| :--------------- |:---------------:| -----:| ----:|
| game_1  |   UOL Hylissang    |  FNC Rekkles | 3 |
| game_1  | UOL Hylissang      |   FNC Rekkles | 5 |
| game_1  | UOL Hylissang      |    FNC Rekkles | 8 |
| game_2  | FNC Rhobalas              |   Trayton | 6 |
| game_2  | FNC Rhobalas          |    Trayton | 15 |
| game_2  | Chap          |    Trayton | 17 |
| game_2  | Chap             |   Trayton | 21 |

Dans cet exemple, sur la première partie (game_1), UOL Hylissang est mort 3 fois dans la partie, tué à chaque fois par FNC Rekkles. Time_kills indique le temps dans le jeu en minutes où le joueur est mort.  


Voici un deuxième exemple: 
    
| Partie_ID  | Victim          | Killer | Time_kills| Gold_type| Gold |
| :----------|:---------------:| ------:| ----:|  ----:| ----:| 
| game_1     | UOL Hylissang   | FNC Rekkles | 3 | Gold_blue | 5k |
| game_1     | UOL Hylissang   | FNC Rekkles | 3 | Gold_red | 4.9k |
| game_1     | UOL Hylissang   | FNC Rekkles | 5 | Gold_blue | 7k |
| game_1     | UOL Hylissang   | FNC Rekkles | 5 | Gold_red | 7.2k |
| game_2     | FNC Rhobalas    | Trayton | 6 | Gold_blue | 7.5k |
| game_2     | FNC Rhobalas    | Trayton | 6 | Gold_red | 7.5k |
| game_2     | FNC Rhobalas    | Trayton | 15 | Gold_blue | 15k |
| game_2     | FNC Rhobalas    | Trayton | 15 | Gold_red | 16k |

Dans ce deuxième exemple, si vous comptez le nombre de mort de FNC Rhobalas, vous aurez des doublons ! Certains colonnes servent de "filtres", ce qui évitent d'avoir trop de colonnes mais ce qui rajoute des lignes.  

    Attention donc aux doublons, en considérant les données utiles pour chaque question ! 

### Combien d'équipes différentes il y a-t-il ? 

### Quel joueur est mort le plus de fois ?

### A quel poste joue ce joueur ? 

### Quel joueur est mort le plus de fois en moyenne par partie ?

### A quel poste joue ce joueur ? 

### Quel champion a le plus haut winrate blueside (Taux de victoire)? Afficher la liste en cas d'égalité.

### Quel champion a le plus haut winrate redside (Taux de victoire)? Afficher la liste en cas d'égalité.

### Quel champion a le plus haut winrate (Taux de victoire)? 

### Afficher une carte des kills en utilisant le package datashader (https://datashader.org/)

### Combien de temps a duré la partie la plus courte ? Donnez la league, l'année, la saison, le type de match et les noms des équipes. 

### Quelle partie a été la plus longue ?  Donnez la league, l'année, la saison, le type de match et les noms des équipes. 

### Affichez la courbe de golddiff de cette partie en précisant les tags d'équipes.
Vous devriez obtenir cette image : ![./images/gold_diff.png](./images/gold_diff.png "Title")

### Quel joueur est mort le plus vite dans une partie (Hors "TooEarly") ? Combien de temps a-t-il tenu ? 

# Question BONUS

### Quel joueur est celui ayant eu le plus de gold au cours d'une partie ? 

### Quel joueur a joué le plus de fois le même champion ? 